In [135]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import warnings

warnings.filterwarnings(action='ignore')
rc('font',family='AppleGothic')
rcParams['axes.unicode_minus'] = False

In [164]:
path = '../data/'

train = pd.read_csv(path+"2_train.csv")
train_errx = pd.read_csv(path+"2_train_errx.csv")
test = pd.read_csv(path+"2_test.csv")
sub = pd.read_csv(path + 'sample_submission.csv')

train.shape, train_errx.shape, test.shape

((2952, 62), (2896, 62), (1008, 61))

In [165]:
# train 과 test를 전처리를 다르게 해야한다.
# train에서는 y 값을 새로운 y를 바꾼거임
# test_X => model => y

In [166]:
def create_new_X(old):
    # 전용면적의 소수점 없애기
    old['전용면적'] = old['전용면적'].apply(int)
    
    # 겹치는 전용면족 행 병합
    columns = list(old.columns)
    columns.remove('전용면적별세대수')
    by_area = old.groupby(columns, as_index=False).agg({'전용면적별세대수': sum})
    
    # 각 전용면적과 그에 상응하는 세대수의 곱
    by_area['면적_세대수_곱']  = by_area['전용면적'] * by_area['전용면적별세대수']
    
    # 각 단지코드당 계산된 실제 총 세대수
    by_area['면적_세대수_곱_총합'] = by_area['면적_세대수_곱'].groupby(by_area['단지코드']).transform('sum')
    
    return by_area

new_train = create_new_X(train)
new_train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공가수,자격유형,임대보증금,임대료,...,J,K,L,M,N,O,등록차량수,전용면적별세대수,면적_세대수_곱,면적_세대수_곱_총합
0,C1000,566,아파트,충청북도,국민임대,39,10.0,A,10803000.0,96030.0,...,0.0,0.0,0.0,0.0,0.0,0.0,481.0,419,16341,23478
1,C1000,566,아파트,충청북도,국민임대,46,10.0,A,14404000.0,129640.0,...,0.0,0.0,0.0,0.0,0.0,0.0,481.0,72,3312,23478
2,C1000,566,아파트,충청북도,국민임대,51,10.0,A,15604000.0,150040.0,...,0.0,0.0,0.0,0.0,0.0,0.0,481.0,75,3825,23478
3,C1013,1308,아파트,광주광역시,국민임대,39,16.0,A,20770000.0,108900.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1665.0,291,11349,59431
4,C1013,1308,아파트,광주광역시,국민임대,46,16.0,A,24665000.0,135960.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1665.0,757,34822,59431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,N2085,1339,아파트,서울특별시,공공임대(분납),84,0.0,A,135531000.0,329790.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1595.0,4,336,68009
1202,N2085,1339,아파트,서울특별시,공공임대(분납),84,0.0,A,136628000.0,332460.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1595.0,4,336,68009
1203,N2085,1339,아파트,서울특별시,공공임대(분납),84,0.0,A,136646000.0,332510.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1595.0,314,26376,68009
1204,N2431,1047,아파트,경상남도,국민임대,36,16.0,A,11217000.0,233330.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1214.0,272,9792,18992


In [167]:
# 여기 공식이 바뀔수있어야한다.
def create_new_y(new_X):
    new_X['전용면적_비례_차량수'] = new_X.apply(lambda row: (row['면적_세대수_곱']) / row['면적_세대수_곱_총합'] * row['등록차량수'], axis=1)
    return new_X['전용면적_비례_차량수']

new_y = create_new_y(new_train)
new_y


0       334.782392
1        67.853821
2        78.363787
3       317.949976
4       975.562080
           ...    
1201      7.880134
1202      7.880134
1203    618.590481
1204    625.920809
1205    588.079191
Name: 전용면적_비례_차량수, Length: 1206, dtype: float64

In [172]:
def get_complex_code(new_X):
    return new_X['단지코드']

In [175]:
def revert_to_old_y(new_y, new_complex_code):
    return pd.concat([new_complex_code, new_y], axis=1).groupby("단지코드").sum()
    

In [176]:
# train으로 revert잘되는지 테스트
new_complex_code = get_complex_code(new_train)
old_y = revert_to_old_y(new_y, new_complex_code)
old_y
# 훈련을 시킨다
# test_X -> create_new_X로 변환시킨다.
# 모델에 predict 시키면 -> test_new_y가 나온다.


,전용면적_비례_차량수
단지코드,
C1000,481.0
C1013,1665.0
C1014,708.0
C1015,579.0
C1017,1246.0
...,...
ERR04_2051,780.0
ERR04_2483,1015.0
N1036,1279.0
